In [ ]:
import shutil
import re

In [ ]:
# BondMachine internal simulator
def runSimbatch():
	! make simbatch > /dev/null
# runSimbatch()

In [ ]:
# BondMachine hardware run
def runBMsim():
	! make deploycollect > /dev/null
	! mv working_dir/bmsim_outputs.seq bmsim_outputs.seq
	! bondmachine -bondmachine-file working_dir/bondmachine.json -list-outputs | wc -l > num_outputs.txt
	# Read the number of outputs
	with open('num_outputs.txt','r') as f:
		num_outputs = int(f.read())
	prefix = getPrefix(data_type)
	if unsigned_base:
		sequence2csvunsigned('bmsim_outputs.seq','bmsim_outputs.csv',num_outputs,data_type,benchcore)
	else:
		sequence2csv('bmsim_outputs.seq','bmsim_outputs.csv',num_outputs,"")
# runBMsim()

In [ ]:
# HLS simulator
def runHLSsim():
	print ("hlsim")
# runHLSsim()

In [ ]:
# Single run test, it executes the entire process and return a DataFrame (df) with the errors
def singleRunTest(runMode,functionFile,neuronUnderTest,neuronSeq,benchcore,data_type,storeBM,runSimbatchTests,runBmsimTests,showLatencyDistribution,singleRunPatch):
	result=[None,None,None,None]  # [dfbsim, dfbm, lbsimhist, lbmsimhist]
	if runMode == "random":
		generateRandom()
	elif runMode == "existent":
		generateExistent(functionFile)
	elif runMode == "fragtester":
		generateNeuron("library", neuronUnderTest, neuronSeq, benchcore, data_type)
	else:
		print (RED + "Unknown run mode: " + runMode + STANDARD)
		exit(1)

	if storeBM:
		# Copy workarea/bondmachine.json to storedbm
		shutil.copy2('working_dir/bondmachine.json', f'storedbm/{neuronUnderTest}_{neuronSeq}.json')

	if runSimbatchTests:
		runSimbatch()
		errors,mse,latency=analyzeSimbatch(benchcore)
		dfbsim=loadBsimRun(mse,latency)
		if singleRunPatch:
			gs=loadBsimGlobalStats()
			PatchBsimGlobalStats(gs, dfbsim)
		result[0]=dfbsim

	if runBmsimTests:
		runBMsim()
		errors,mse,latency=analyzeBMsim(benchcore)
		dfbm=loadBMsimRun(mse,latency)
		if singleRunPatch:
			gs=loadBmsimGlobalStats()
			PatchBmsimGlobalStats(gs, dfbm)
		result[1]=dfbm

	# runHLSsim()

	if showLatencyDistribution and runSimbatchTests:
		lbsimhist=latencyHistogram('simbatch_outputs.csv','bsimlatency','Simbatch Latency Distribution')
		result[2]=lbsimhist

	if showLatencyDistribution and runBmsimTests:
		lbmsimhist=latencyHistogram('bmsim_outputs.csv','bmlatency','BM Latency Distribution')
		result[3]=lbmsimhist

	return result	

In [ ]:

def fullRunTest(debug,benchcore,data_type,runSimbatchTests,runBmsimTests,regexFilter):
	# Iterate al the neurons in the library and test them singly, patching the global statistics
	gs=loadGlobalStats()

	for neuronFile in os.listdir("library"):
		if not neuronFile.endswith(".basm"):
			continue
		if regexFilter != "":
			match = re.search(regexFilter, neuronFile)
			if not match:
				continue
		neuron = neuronFile.split(".")[0]
		# generateRandom()
		# generateExistent()

		if debug:
			print("Evaluating neuron: "+neuron)

		# Simbatch tests
		if runSimbatchTests and not(any(gs.index.str.startswith(neuron)) and gs.loc[gs.index.str.startswith(neuron), 'bsimoccurrences'].max() > 0):
			seq=getNeuronSeq("library", neuron, data_type)
			if seq!="0":
				for i in range(0, int(seq)):
					if generateNeuron("library", neuron, str(i), benchcore, data_type):
						runSimbatch()
						errors,mse,latency=analyzeSimbatch(benchcore)
						df=loadBsimRun(mse,latency)
						gs=PatchBsimGlobalStats(gs, df)
					else:
						print(f"{RED}Error generating neuron {neuron} with sequence {seq}{STANDARD}")
			else:
				placeHolderNeuron("library", neuron)

				df=loadBsimRun(0.0,False)
				gs=PatchBsimGlobalStats(gs, df)
		else:
			if debug:
				print(f"{CYAN}Skipping neuron {neuron} for Simbatch tests{STANDARD}")

			# BMsim tests

		# BMsim tests
		if runBmsimTests and not(any(gs.index.str.startswith(neuron)) and gs.loc[gs.index.str.startswith(neuron), 'bmoccurrences'].max() > 0):
			seq=getNeuronSeq("library", neuron, data_type)
			if seq!="0":
				for i in range(0, int(seq)):
					if generateNeuron("library", neuron, str(i), benchcore, data_type):
						runBMsim()
						errors,mse,latency=analyzeBMsim(benchcore)
						df=loadBMsimRun(mse,latency)
						gs=PatchBMsimGlobalStats(gs, df)
					else:
						print(f"{RED}Error generating neuron {neuron} with sequence {seq}{STANDARD}")
			else:
				placeHolderNeuron("library", neuron)

				df=loadBMsimRun(0.0,False)
				gs=PatchBMsimGlobalStats(gs, df)
		else:
			if debug:
				print(f"{CYAN}Skipping neuron {neuron} for BMsim tests{STANDARD}")
				
	return gs
